# Querying the Genesis database

In [1]:
import h5py
import numpy as np
from pylab import *
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
from astropy import units, constants
from matplotlib.patches import Wedge
from matplotlib.collections import PatchCollection

### Helper functions

In [2]:
def mass_to_size(mass, slim=[5,500], mlim=[1,10]):
    s1, s2= slim
    m1, m2= mlim
    mass=np.asarray(mass)
    size= s1 + (s2-s1)* np.log10(mass/m1) / np.log10(m2/m1)
    try:
        size[mass<m1]= s1
        size[mass>m2]= s2
    except TypeError:
        pass

    return size

## Load the genesis database from an hdf5 file

#### Models publicly availabel at http://eos-nexus.org/genesis-database/ 

In [3]:
hf= h5py.File('genesis_all.hdf5','r')

## Read in data

In [4]:
example= hf['Gen-HM']['run_34']

In [5]:
particle_i= example['collisions']['particle_i'][()]
particle_j= example['collisions']['particle_j'][()]
collision_time= example['collisions']['collision_time'][()]
particles = np.append(particle_i, particle_j)

sma = example['snapshots']['sma'][()] 
per = np.sqrt(sma**3)
time= example['snapshots']['t'][()]
inc = example['snapshots']['inc'][()]
mass= example['snapshots']['mass'][()]
ecc = example['snapshots']['ecc'][()]
bbid = example['snapshots']['id'][()]

In [6]:
unique_time = np.unique(time)

In [7]:
planet_ids = []
for key, planet in example['planets'].items():
    planet_ids.append(planet['id'][()])

In [8]:
cmap = cm.get_cmap('viridis', len(planet_ids))
vcolors = []
for i in range(cmap.N):
    rgb = cmap(i)[:3]
    vcolors.append(matplotlib.colors.rgb2hex(rgb))
vcolors = np.array(vcolors)

## Creating the orbits and when collisions occur

In [61]:
interptime = np.append(np.linspace(0, collision_time[25],500), 
                       np.linspace(collision_time[25], collision_time[35]-0.1,700))
interptime = np.append(interptime,
                       np.linspace(collision_time[35], 2, 900))

times = np.full((len(np.unique(particles)), len(interptime)), 
                 interptime)
x_grid = np.full((len(np.unique(particles)), len(interptime)), 0.0)
y_grid = np.full((len(np.unique(particles)), len(interptime)), 0.0)
m_grid = np.full((len(np.unique(particles)), len(interptime)), 0.0)
a_grid = np.full((len(np.unique(particles)), len(interptime)), 0.0)
p_grid = np.full((len(np.unique(particles)), len(interptime)), 0.0)
id_grid= np.full((len(np.unique(particles)), len(interptime)), 0.0)
coll_grid = np.full((len(np.unique(particles)), len(interptime)), False, dtype=bool)
color_grid = np.full((len(np.unique(particles)), len(interptime)), '', dtype='U10')

###########################################################
###########################################################
###########################################################
i = 0
where_i = []
for key,planet in example['planets'].items():
    for bb in planet['bb'][()]:
        if bb == planet['id'][()]:
            where_i.append(i)
        i += 1
###########################################################
i,r = 0,0
order = [0, 5, 3, 1, 4, 2]
vcolors = vcolors[order]

for key, planet in example['planets'].items():
    planetnum = planet['id'][()]
    
    where = np.where(bbid==planetnum)[0]
    a = sma[where][-1] + 0.0
    p = np.sqrt(a**3)
    
    coremass = mass[np.where(bbid==planetnum)[0][0]]
    tempcolor = vcolors[r]

    for bb in planet['bb'][()]:
        if bb != planetnum:
            pt = np.where((particle_i==bb) | (particle_j==bb))[0][-1]
            k  = np.where(interptime<=collision_time[pt])[0][-1]
            b  = np.where(bbid==bb)[0][-1]
            a_grid[i][k] =  sma[b]
            q = interptime>=collision_time[pt]
            m_grid[where_i[r]][q] += np.full(len(interptime[q]), mass[b])
            coll_grid[i][k] = True
            color_grid[i][k] = tempcolor
        else:
            for ut in range(len(np.unique(time))-1):
                q = ((interptime>=np.unique(time)[ut]) &
                     (interptime<np.unique(time)[ut+1]))

                k = np.where((time==np.unique(time)[ut]) &
                             (bbid==planet['id'][()]))[0]

                m_grid[i][q] += np.full(len(interptime[q]), coremass)
                color_grid[i][q] = tempcolor

        p_grid[i] = np.full(len(interptime), p)
        id_grid[i] = r

        factor = 2*np.pi*interptime / p

        x_grid[i] = a*np.cos(factor)
        y_grid[i] = a*np.sin(factor)

        i += 1
    r += 1

In [62]:
indexing = id_grid.reshape(id_grid.shape[0]*id_grid.shape[1])
q = np.isnan(indexing) == False
masses = m_grid.reshape(m_grid.shape[0]*m_grid.shape[1])
times  = times.reshape(times.shape[0]*times.shape[1])
smas   = a_grid.reshape(a_grid.shape[0]*a_grid.shape[1])
x = x_grid.reshape(x_grid.shape[0]*x_grid.shape[1])
y = y_grid.reshape(y_grid.shape[0]*y_grid.shape[1])
colls = coll_grid.reshape(coll_grid.shape[0]*coll_grid.shape[1])
plot_colors = color_grid.reshape(color_grid.shape[0]*color_grid.shape[1])

## Get temperature regions

In [63]:
def loc(t):
    return np.sqrt(L/(16*np.pi*constants.sigma_sb*t**4)).to(units.AU).value

In [64]:
L = 0.072*units.Lsun

# water, CO
temps = [[373, 273]*units.Kelvin, [56+273, -31+273]*units.Kelvin]

h2oinner = loc(temps[0][0])
h2oouter = loc(temps[0][1])

coinner = loc(temps[1][0])
coouter = loc(temps[1][1])

## Making the animation

In [65]:
plt.rcParams['axes.formatter.min_exponent'] = 3
plt.rcParams['font.size'] = 16

f, ax = plt.subplots(figsize=(10,8), sharey=True)
#ax.set_facecolor('k')
plt.plot(0,0,'*', ms=10, c='k', markeredgecolor='k')
lim = 1.05
plt.xlim(-lim, lim)
plt.ylim(-lim, lim)
plt.xlabel('Distance [AU]', fontsize=18)
plt.ylabel('Distance [AU]', fontsize=18)
############################################################
circx = np.linspace(0, 2*np.pi, 400)
############################################################
## ADDING BACKGROUND RINGS
water_radii = np.linspace(h2oinner, h2oouter, 100)
co_radii = np.linspace(coinner, coouter, 100)

patches = [Wedge((0, 0), h2oouter, 0, 360, width=h2oouter-h2oinner),
           Wedge((0, 0), coouter, 0, 360, width=coouter-coinner)]

p = PatchCollection(patches, alpha=0.4, cmap='Set1')
p.set_array(np.array([0, 1]))
ax.add_collection(p)
############################################################
############################################################
## ADDING SIDE REFERENCES
plt.subplots_adjust(bottom=0.1, right=0.8, top=0.9)
ref_ax = f.add_axes([0.83, 0.1, 0.13, 0.8])
references = [1.0, 2.0, 4.0, 8.0]
locs = np.linspace(-np.nanmax(smas)+np.nanmax(smas)/4.0,
            np.nanmax(smas)-np.nanmax(smas)/4.0,4)

for bb in range(len(references)):
    ref_ax.scatter(0.2, locs[bb], s=mass_to_size(references[bb])*3, 
                   c='w', edgecolor='k', linewidth=2)
    ref_ax.text(1.4, locs[bb]-0.03, str(int(references[bb])))
    
ref_ax.set_ylim(-np.nanmax(smas)-0.05, np.nanmax(smas)+0.05)
ref_ax.set_yticklabels([])
ref_ax.set_yticks([])
ref_ax.set_xticklabels([])
ref_ax.set_xticks([])
ref_ax.axis('off')
ref_ax.set_xlim(-1,3)
ref_ax.set_title('Mass [M$_{Earth}$]', fontsize=18)
############################################################
############################################################
## THE ANIMATION
frames = []
time_offset = 0

for i in range(len(np.unique(times))-1):
    q = ((times == np.unique(times)[i]) & (masses!=0) &
         (plot_colors!=''))

    w = times <= np.unique(times)[i]
    newring = np.where(colls[w] == True)[0]

    orbit_frames = []
    if len(smas[w][newring]) > 0:
        for s, r in zip(smas[w][newring], indexing[w][newring]):
            of = ax.plot(s*np.cos(circx), 
                         s*np.sin(circx),
                         c=vcolors[int(r)], 
                         alpha=0.8,
                         linewidth=3)
            orbit_frames.append(of)
            
    frame = ax.scatter(x[q], y[q], 
                       s=mass_to_size(masses[q])*3, 
                       c=plot_colors[q],
                       marker='o',
                       animated=True,
                       edgecolor='k', linewidth=0.6, zorder=100)
    #frame = ax.scatter(np.linspace(-0.2,0.2,4)*i, np.linspace(0,1,4),animated=True)
    text = ax.text(x=-1.45, y=0.95, 
                   s='t= {:.3f} Myr'.format(np.unique(times)[i] ), 
                   animated=True, 
                   transform=plt.gca().transAxes)
    
    frames.append(np.append([frame, text], orbit_frames))
    
############################################################
############################################################
plt.close()   
ani = animation.ArtistAnimation(f, frames, 
                                interval=25, blit=True, 
                                repeat=False)
############################################################
############################################################
# Show the animation
HTML(ani.to_html5_video())

In [66]:
Writer = animation.writers['ffmpeg']
writer = Writer(fps=30, metadata=dict(artist='Adina Feinstein'), bitrate=1800)
ani.save('/Users/arcticfox/Desktop/feinstein_genhm_sim.mp4', writer=writer)

CalledProcessError: Command '['ffmpeg', '-f', 'rawvideo', '-vcodec', 'rawvideo', '-s', '720x576', '-pix_fmt', 'rgba', '-r', '30', '-loglevel', 'error', '-i', 'pipe:', '-vcodec', 'h264', '-pix_fmt', 'yuv420p', '-b', '1800k', '-metadata', 'artist=Adina Feinstein', '-y', '/Users/arcticfox/Desktop/feinstein_genhm_sim.mp4']' returned non-zero exit status 255.